In [1]:
import numpy as np
import qutip as qt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.circuit.library import QFT
from random import randint
import matplotlib.pyplot as plt
from time import time
import pickle


from tools.classical import *
from tools.quantum import *
from boltzmann import *
from op_fourier_trafo import *

# len([gate for gate in tr_circ0.data if 'cx' in str(gate)])

In [2]:
# # Unpickle the object
# with open('data.pkl', 'rb') as f:
#     data = pickle.load(f)


In [3]:

# num_energy_bits = 6
# num_qubits = 3
# qr_boltzmann = QuantumRegister(1, name='qr_boltz')
# qr_energy = QuantumRegister(num_energy_bits, name='qr_energy')

# cr_boltzmann = ClassicalRegister(1, name='cr_boltz')
# cr_energy = ClassicalRegister(num_energy_bits, name='cr_energy')

# bithandler = BitHandler([cr_boltzmann, cr_energy])

# circ0 = QuantumCircuit(qr_boltzmann, qr_energy, cr_boltzmann, cr_energy)
# circ1 = QuantumCircuit(qr_boltzmann, qr_energy, cr_boltzmann, cr_energy)

# boltz_circ = lookup_table_boltzmann(num_energy_bits)

# #* By hand
# t0 = time()
# inverse_boltz_circ = reverse_lookup_table_boltzmann(num_energy_bits)
# circ0.compose(boltz_circ, [qr_boltzmann[0], *list(qr_energy)], inplace=True)
# circ0.compose(inverse_boltz_circ, [qr_boltzmann[0], *list(qr_energy)], inplace=True)
# tr_circ0 = transpile(circ0, basis_gates=['cx', 'u3'], optimization_level=3)
# t1 = time()
# #* By Qiskit
# t2 = time()
# qiskit_inverse_boltz_circ = boltz_circ.inverse()
# circ1.compose(boltz_circ, [qr_boltzmann[0], *list(qr_energy)], inplace=True)
# circ1.compose(qiskit_inverse_boltz_circ, [qr_boltzmann[0], *list(qr_energy)], inplace=True)
# tr_circ1 = transpile(circ1, basis_gates=['cx', 'u3'], optimization_level=3)
# t3 = time()

# op0 = Operator(circ0)
# op1 = Operator(circ1)
# dist_to_identity0 = np.linalg.norm(op0.data - np.eye(2**(1+num_energy_bits)))
# dist_to_identity1 = np.linalg.norm(op1.data - np.eye(2**(1+num_energy_bits)))

# print(f"By hand: {t1-t0:.3f} s")
# print(dist_to_identity0)
# print(f"By Qiskit: {t3-t2:.3f} s")
# print(dist_to_identity1)

In [4]:
np.random.seed(667)
num_qubits = 3
num_energy_bits = 6
bohr_bound = 2 ** (-num_energy_bits + 1) #!
eps = 0.05
sigma = 10
eig_index = 0
T = 1
shots = 100

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)
rescaled_coeff = hamiltonian.rescaled_coeffs
# Corresponding Trotter step circuit
trotter_step_circ = trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeff, symbreak=True)
hamiltonian.trotter_step_circ = trotter_step_circ

Original spectrum:  [-3.5    -2.9921 -2.3725 -1.5     1.6025  2.1     2.8895  3.7725]
Ideal spectrum:  [-0.0071  0.0255  0.0653  0.1214  0.3209  0.3529  0.4036  0.4604]
Nonrescaled coefficients:  [0.9 0.7 0.9 1. ]
Rescaled coefficients:  [0.05785714 0.045      0.05785714 0.06428571]


In [5]:
qr_energy = QuantumRegister(num_energy_bits, name='qr_energy')
qr_sys = QuantumRegister(num_qubits, name='qr_sys')
cr_energy = ClassicalRegister(num_energy_bits, name='cr_energy')
cr_sys = ClassicalRegister(num_qubits, name='cr_sys')

bithandler = BitHandler([cr_energy, cr_sys])

circ0 = QuantumCircuit(qr_energy, qr_sys, cr_energy, cr_sys)
circ1 = QuantumCircuit(qr_energy, qr_sys, cr_energy, cr_sys)

#* Ini
# if sigma != 0.:
#     prep_circ = brute_prepare_gaussian_state(num_energy_bits, sigma)
#     circ.compose(prep_circ, [*list(qr_energy)], inplace=True)
# else:  # Conventional QPE
#     circ.h(qr_energy)
    
# System prep, initial state = eigenstate
initial_state = np.array([1] + [0] * (2**num_qubits - 1))

#* OFT
jump_op = Operator(Pauli('X'))
oft_circ = operator_fourier_circuit(jump_op, num_qubits, num_energy_bits, hamiltonian, 
                                    initial_state=initial_state)



Energy before jump: 0.45571428571428574
Jump applied to 1th qubit
Energy after jump: 0.09571428571428571
Energy jump: -0.36000000000000004


In [6]:
inverse_trotter_step_circ = inverse_trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeff, symbreak=True)
hamiltonian.trotter_step_circ = inverse_trotter_step_circ

In [7]:
#* By hand
t0 = time()
inverse_oft_circ = inverse_operator_fourier_transform(jump_op, num_qubits, num_energy_bits, hamiltonian)
t1 = time()
print(f"Inverse by hand: {t1-t0:.3f} s")

with open(f'data/inverse_oft_byhand_{num_qubits}{num_energy_bits}.pkl', 'wb') as f:
    pickle.dump(inverse_oft_circ, f)

Jump applied to 1th qubit
Inverse by hand: 2.482 s


In [8]:
t0 = time()
inverse_oft_op = Operator(inverse_oft_circ)
t1 = time()
print(f"Hattened: {t1-t0:.3f} s")

with open(f'data/inverse_oft_byhand_op_{num_qubits}{num_energy_bits}.pkl', 'wb') as f:
    pickle.dump(inverse_oft_op, f)

Hattened: 544.274 s


In [9]:
#* By Qiskit
t0 = time()
qiskit_inverse_oft_circ = oft_circ.inverse()
t1 = time()
print(f"Inverse by Qiskit: {t1-t0:.3f} s")

with open(f'data/inverse_oft_qiskit_{num_qubits}{num_energy_bits}.pkl', 'wb') as f:
    pickle.dump(qiskit_inverse_oft_circ, f)

Inverse by Qiskit: 7.151 s


In [10]:
t0 = time()
qiskit_inverse_oft_op = Operator(qiskit_inverse_oft_circ)
t1 = time()
print(f"Hattened Qiskit {t1-t0:.3f} s")

with open(f'data/inverse_oft_qiskit_op_{num_qubits}{num_energy_bits}.pkl', 'wb') as f:
    pickle.dump(qiskit_inverse_oft_op, f)

Hattened Qiskit 552.489 s


In [11]:
oft_op = Operator(oft_circ)

In [12]:
dist_to_identity_byhand = np.linalg.norm(inverse_oft_op.data @ oft_op.data - np.eye(2**(num_qubits + num_energy_bits)))
dist_to_identity_qiskit = np.linalg.norm(qiskit_inverse_oft_op.data @ oft_op.data - np.eye(2**(num_qubits + num_energy_bits)))
print(f'Distance to identity by hand: {dist_to_identity_byhand}')
print(f'Distance to identity Qiskit: {dist_to_identity_qiskit}')

Distance to identity by hand: 3.266587103381993e-11
Distance to identity Qiskit: 3.266587103381993e-11


In [13]:
print((inverse_oft_op.data @ oft_op.data).round(3))

[[ 1.-0.j  0.+0.j -0.+0.j ... -0.-0.j  0.-0.j  0.-0.j]
 [ 0.+0.j  1.-0.j  0.-0.j ... -0.+0.j  0.-0.j -0.+0.j]
 [-0.-0.j -0.-0.j  1.-0.j ...  0.+0.j -0.-0.j  0.-0.j]
 ...
 [-0.+0.j  0.-0.j  0.-0.j ...  1.-0.j -0.+0.j -0.+0.j]
 [ 0.+0.j -0.+0.j  0.+0.j ... -0.-0.j  1.-0.j  0.-0.j]
 [-0.+0.j -0.+0.j -0.+0.j ...  0.-0.j -0.+0.j  1.-0.j]]


In [14]:
np.random.seed(666)
randstate_better = np.zeros(2**(num_qubits+num_energy_bits))
randstate_better[np.random.choice(2**num_qubits, 2**num_qubits//2, replace=False)] = 1
randstate_better /= np.linalg.norm(randstate_better)

end_state = inverse_oft_op.data @ oft_op.data @ randstate_better
end_state_qiskit = qiskit_inverse_oft_op.data @ oft_op.data @ randstate_better
print(f'By hand fidelity: {qt.fidelity(qt.Qobj(randstate_better), qt.Qobj(end_state))}')
print(f'Qiskit fidelity: {qt.fidelity(qt.Qobj(randstate_better), qt.Qobj(end_state_qiskit))}')


By hand fidelity: 0.9999999999998774
Qiskit fidelity: 0.9999999999998774


In [15]:
t0 = time()
tr_inverse_byhand = transpile(inverse_oft_circ, basis_gates=['cx', 'u3'], optimization_level=3)
t1 = time()
print(f'Transpiled the byhand circuit: {t1-t0:.3f} s')

/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


Transpiled the byhand circuit: 96.404 s


In [16]:
t0 = time()
tr_inverse_qiskit = transpile(qiskit_inverse_oft_circ, basis_gates=['cx', 'u3'], optimization_level=3)
t1 = time()
print(f'Transpiled the Qiskit circuit: {t1-t0:.3f} s')

Transpiled the Qiskit circuit: 116.514 s


In [17]:
# circ1.compose(oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)
# circ1.compose(qiskit_inverse_oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)
# tr_circ1 = transpile(circ1, basis_gates=['cx', 'u3'], optimization_level=3)

In [18]:
# # inverse_oft_op = Operator(inverse_oft_circ)
# circ0.compose(oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)
# circ0.compose(inverse_oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)
# tr_circ0 = transpile(circ0, basis_gates=['cx', 'u3'], optimization_level=3)

In [19]:

# op0 = Operator(circ0)
# op1 = Operator(circ1)


In [20]:
# dist_to_identity0 = np.linalg.norm(op0.data - np.eye(2**(1+num_energy_bits)))
# dist_to_identity1 = np.linalg.norm(op1.data - np.eye(2**(1+num_energy_bits)))

# print(f"By hand: {t1-t0:.3f} s")
# print(dist_to_identity0)
# print(f"By Qiskit: {t3-t2:.3f} s")
# print(dist_to_identity1)